In [6]:
import os
import sys

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from prefix_span import PrefixSpan
from js_distance import JS
from sequence_generator import SequenceGenerator

In [ ]:
seq = SequenceGenerator(csvfile='~/db/csvfile.csv', jsThreshold=0.01)
    data = seq.generate_sequence()
    print(len(data))